## Prerequisites

In [1]:
from paraview.simple import *
from ipyparaview.widgets import PVDisplay
from ipywidgets import interact, widgets
from IPython.display import Javascript, HTML
import json
import random
import math

from ipywidgets import FileUpload

In [2]:
from IPython.core.display import display, HTML
    
def f(test = "Show"):
    
    toggle_code_str = '''
    <form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Toggle"></form>
    '''
    toggle_code_str = toggle_code_str.replace("Toggle", "Toggle "+test+" function")
    

    toggle_code_prepare_str = '''
        <script>
        
        function code_toggle() {
            if ($('div.cell.code_cell.rendered.selected div.input').css('display')!='none'){
                $('div.cell.code_cell.rendered.selected div.input').hide();
            } else {
                $('div.cell.code_cell.rendered.selected div.input').show();
            }
        }
        </script>

    '''
    display(HTML(toggle_code_prepare_str + toggle_code_str))
    
def hide_solution():
    toggle_code_str = '''
    <form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Solution"></form>
    '''
    display(HTML(toggle_code_str))

f('toggle')

In [3]:
HTML("""<link rel="stylesheet" href="fuzzyContourTrees.css" type="text/css">""")

In [4]:
%%javascript
require.config({ 
     paths: { 
     d3: 'https://d3js.org/d3.v5.min',
     jQuery: 'https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquery.min'    
}});

<IPython.core.display.Javascript object>

## Alignment Computation

In [5]:
# ----------------------------------------------------------------
# setup the data processing pipelines
# ----------------------------------------------------------------

# create a new 'TTK CinemaReader'
tTKCinemaReader1 = TTKCinemaReader(DatabasePath='/home/data/heatedCylinder3dEnsemble_scalarfields.cdb')

# create a new 'TTK CinemaQuery'
tTKCinemaQuery1 = TTKCinemaQuery(InputTable=tTKCinemaReader1)

# example SQL statement for time dependent alignments
'''tTKCinemaQuery1.SQLStatement = """SELECT * FROM InputTable0 WHERE sim=1 AND 
    (time=4.2 OR time=4.25 OR time=4.3 OR time=4.35 OR time=4.4 OR time=4.45 OR time=4.1)
    ORDER BY time"""'''

# example SQL statement for non time dependent alignments
tTKCinemaQuery1.SQLStatement = """SELECT * FROM InputTable0 WHERE time=3.8"""

# create a new 'TTK CinemaProductReader'
tTKCinemaProductReader1 = TTKCinemaProductReader(Input=tTKCinemaQuery1)

# create a new 'TTK ForEach'
tTKForEach1 = TTKForEach(Input=tTKCinemaProductReader1)
tTKForEach1.InputArray = ['POINTS', 'Scalar']

# create a new 'Merge Blocks'
mergeBlocks1 = MergeBlocks(Input=tTKForEach1)

# create a new 'Tetrahedralize'
tetrahedralize1 = Tetrahedralize(Input=mergeBlocks1)

# create a new 'TTK PersistenceDiagram'
tTKPersistenceDiagram1 = TTKPersistenceDiagram(Input=tetrahedralize1)
tTKPersistenceDiagram1.ScalarField = ['POINTS', 'Scalar']
tTKPersistenceDiagram1.InputOffsetField = ['POINTS', 'Scalar']

# create a new 'Threshold'
threshold1 = Threshold(Input=tTKPersistenceDiagram1)
threshold1.Scalars = ['CELLS', 'PairIdentifier']
threshold1.ThresholdRange = [0.0, 9999999.0]

# create a new 'Threshold'
threshold2 = Threshold(Input=threshold1)
threshold2.Scalars = ['CELLS', 'Persistence']
threshold2.ThresholdRange = [0.05, 99999999999.0]

# create a new 'TTK TopologicalSimplification'
tTKTopologicalSimplification1 = TTKTopologicalSimplification(Domain=tetrahedralize1,
    Constraints=threshold2)
tTKTopologicalSimplification1.ScalarField = ['POINTS', 'Scalar']
tTKTopologicalSimplification1.InputOffsetField = ['POINTS', 'Scalar']
tTKTopologicalSimplification1.VertexIdentifierField = ['POINTS', 'CriticalType']

# create a new 'TTK Merge and Contour Tree (FTM)'
tTKMergeandContourTreeFTM1 = TTKMergeandContourTreeFTM(Input=tTKTopologicalSimplification1)
tTKMergeandContourTreeFTM1.ScalarField = ['POINTS', 'Scalar']
tTKMergeandContourTreeFTM1.InputOffsetField = ['POINTS', 'Scalar']

# find source
tTKMergeandContourTreeFTM1_1 = FindSource('TTKMergeandContourTreeFTM1')

# create a new 'TTK ArrayEditor'
tTKArrayEditor1 = TTKArrayEditor(Target=OutputPort(tTKMergeandContourTreeFTM1_1,1),
    Source=tetrahedralize1)
tTKArrayEditor1.EditorMode = 'Add Arrays from Source'
tTKArrayEditor1.SourceFieldDataArrays = ['_ttk_IterationInfo']
tTKArrayEditor1.TargetArray = ['CELLS', 'SegmentationId']

# create a new 'TTK BlockAggregator'
tTKBlockAggregator1 = TTKBlockAggregator(Input=tTKArrayEditor1)

# create a new 'TTK EndFor'
tTKEndFor1 = TTKEndFor(Data=tTKBlockAggregator1,
    For=tTKForEach1)

In [6]:
# create a new 'TTK ContourTreeAlignment'

tTKContourTreeAlignment1 = TTKContourTreeAlignment(Input=tTKEndFor1,

    ExportPath='/home/wetzels/alignmentJSON')

tTKContourTreeAlignment1.ScalarField = ['POINTS', 'Scalar']

tTKContourTreeAlignment1.Regionsizearray = ['CELLS', 'RegionSize']

tTKContourTreeAlignment1.SegmentationIDarray = ['CELLS', 'SegmentationId']

tTKContourTreeAlignment1.Matchovertime = 1

tTKContourTreeAlignment1.ExportalignmentandtreesasJSONfile = 1

tTKContourTreeAlignment1.Seed = 12



# create a new 'TTK PlanarGraphLayout'
tTKPlanarGraphLayout1 = TTKPlanarGraphLayout(Input=tTKContourTreeAlignment1)
tTKPlanarGraphLayout1.SequenceArray = ['POINTS', 'Scalar']
tTKPlanarGraphLayout1.SizeArray = ['POINTS', 'BranchIDs']
tTKPlanarGraphLayout1.UseBranches = 1
tTKPlanarGraphLayout1.BranchArray = ['POINTS', 'BranchIDs']
tTKPlanarGraphLayout1.LevelArray = ['POINTS', 'BranchIDs']

# create a new 'Calculator'
calculator1 = Calculator(Input=tTKPlanarGraphLayout1)
calculator1.CoordinateResults = 1
calculator1.Function = 'iHat*Layout_Y + jHat*Scalar*3'

In [7]:
# Create a new 'Render View'
renderView1 = CreateView('RenderView')
renderView1.ViewSize = [1221, 769]
renderView1.InteractionMode = '2D'
renderView1.AxesGrid = 'GridAxes3DActor'
renderView1.CenterOfRotation = [3.1875000596046448, 1.1477879285812378, 0.0]
renderView1.StereoType = 'Crystal Eyes'
renderView1.CameraPosition = [3.1875000596046448, 1.1477879285812378, 6.98226681753297]
renderView1.CameraFocalPoint = [3.1875000596046448, 1.1477879285812378, 0.0]
renderView1.CameraFocalDisk = 1.0
renderView1.CameraParallelScale = 1.8071436303648998

In [8]:
# show data from calculator1
calculator1Display = Show(calculator1, renderView1, 'UnstructuredGridRepresentation')

# get color transfer function/color map for 'Scalar'
scalarLUT = GetColorTransferFunction('Scalar')
scalarLUT.RGBPoints = [0.0, 0.231373, 0.298039, 0.752941, 0.3825959861278534, 0.865003, 0.865003, 0.865003, 0.7651919722557068, 0.705882, 0.0156863, 0.14902]
scalarLUT.ScalarRangeInitialized = 1.0

# get opacity transfer function/opacity map for 'Scalar'
scalarPWF = GetOpacityTransferFunction('Scalar')
scalarPWF.Points = [0.0, 0.0, 0.5, 0.0, 0.7651919722557068, 1.0, 0.5, 0.0]
scalarPWF.ScalarRangeInitialized = 1

# trace defaults for the display properties.
calculator1Display.Representation = 'Surface'
calculator1Display.ColorArrayName = ['POINTS', 'Scalar']
calculator1Display.LookupTable = scalarLUT
calculator1Display.LineWidth = 3.0
calculator1Display.OSPRayScaleArray = 'BranchIDs'
calculator1Display.OSPRayScaleFunction = 'PiecewiseFunction'
calculator1Display.SelectOrientationVectors = 'BranchIDs'
calculator1Display.ScaleFactor = 0.07651919722557068
calculator1Display.SelectScaleArray = 'BranchIDs'
calculator1Display.GlyphType = 'Arrow'
calculator1Display.GlyphTableIndexArray = 'BranchIDs'
calculator1Display.GaussianRadius = 0.003825959861278534
calculator1Display.SetScaleArray = ['POINTS', 'BranchIDs']
calculator1Display.ScaleTransferFunction = 'PiecewiseFunction'
calculator1Display.OpacityArray = ['POINTS', 'BranchIDs']
calculator1Display.OpacityTransferFunction = 'PiecewiseFunction'
calculator1Display.DataAxesGrid = 'GridAxesRepresentation'
calculator1Display.PolarAxes = 'PolarAxesRepresentation'
calculator1Display.ScalarOpacityFunction = scalarPWF
calculator1Display.ScalarOpacityUnitDistance = 0.2975916659753401

# init the 'PiecewiseFunction' selected for 'ScaleTransferFunction'
calculator1Display.ScaleTransferFunction.Points = [0.0, 0.0, 0.5, 0.0, 8.0, 1.0, 0.5, 0.0]

# init the 'PiecewiseFunction' selected for 'OpacityTransferFunction'
calculator1Display.OpacityTransferFunction.Points = [0.0, 0.0, 0.5, 0.0, 8.0, 1.0, 0.5, 0.0]

# setup the color legend parameters for each legend in this view

# get color legend/bar for scalarLUT in view renderView1
scalarLUTColorBar = GetScalarBar(scalarLUT, renderView1)
scalarLUTColorBar.Title = 'Scalar'
scalarLUTColorBar.ComponentTitle = ''

# set color bar visibility
scalarLUTColorBar.Visibility = 1

# show color legend
calculator1Display.SetScalarBarVisibility(renderView1, True)

# ----------------------------------------------------------------
# setup color maps and opacity mapes used in the visualization
# note: the Get..() functions create a new object, if needed
# ----------------------------------------------------------------

# ----------------------------------------------------------------
# finally, restore active source
SetActiveSource(calculator1)
# ----------------------------------------------------------------

In [9]:
disp = PVDisplay(renderView1)
display(disp)

PVDisplay(resolution=(1221, 769))

In [10]:
import paraview.servermanager
alignment_vtk = paraview.servermanager.Fetch(tTKContourTreeAlignment1)
input_MB = paraview.servermanager.Fetch(tTKEndFor1)

In [11]:
#import numpy as np

nAlignmentNodes = alignment_vtk.GetNumberOfPoints()
nAlignmentEdges = alignment_vtk.GetNumberOfCells()
n = input_MB.GetNumberOfBlocks()
nVertices = []
for i in range(0,n):
    nVertices.append(input_MB.GetBlock(i).GetNumberOfPoints())

print(nAlignmentNodes)
print(nAlignmentEdges)
print(n)
print(nVertices)

vertexIDs = alignment_vtk.GetPointData().GetArray("VertexIDs")

print(vertexIDs)

def alignmentToJSON():
    upEdges = []
    downEdges = []
    for i in range(0,nAlignmentNodes):
        upEdges.append([])
        downEdges.append([])

    alignmentIDs = []
    for i in range(n):
        vertices_i = []
        for i in range(0,nVertices[i]):
            vertices_i.append(-1)
        alignmentIDs.append(vertices_i)
    

    for i in range(nAlignmentEdges):
        id1 = alignment_vtk.GetCell(i).GetPointId(0)
        id2 = alignment_vtk.GetCell(i).GetPointId(1)
        v1 = alignment_vtk.GetPointData().GetArray("Scalar").GetValue(id1)
        v2 = alignment_vtk.GetPointData().GetArray("Scalar").GetValue(id2)
        if(v1 > v2):
            downEdges[id1].append(i)
            upEdges[id2].append(i)
        else:
            upEdges[id1].append(i)
            downEdges[id2].append(i)
            
    res = []
    
    #==================================================================================================================
    # output original trees as JSON

    textJSON = "{\n"

    textJSON += "  \"nodes\": [\n"

    for i in range(0,nAlignmentNodes):
        textJSON += "    {"
        textJSON += "\"scalar\": " + str(alignment_vtk.GetPointData().GetArray("Scalar").GetValue(i)) + ", "
        textJSON += "\"frequency\": " + str(alignment_vtk.GetPointData().GetArray("Frequency").GetValue(i)) + ", "
        for j in range(0,n):
            if(vertexIDs.GetComponent(i, j) >= 0):
                alignmentIDs[j][int(vertexIDs.GetComponent(i, j))] = i
        textJSON += "\"segmentationIDs\": ["
        for j in range(0,n):
            textJSON += str(alignment_vtk.GetPointData().GetArray("segmentationIDs").GetComponent(i, j))
            if(j < n - 1):
                textJSON += ","
        textJSON += "], "
        textJSON += "\"upEdgeIDs\": ["
        for j in range(0,len(upEdges[i])):
            textJSON += str(upEdges[i][j])
            if(j < len(upEdges[i]) - 1):
                textJSON += ","
        textJSON += "], ";
        textJSON += "\"downEdgeIDs\": [";
        for j in range(0,len(downEdges[i])):
            textJSON += str(downEdges[i][j])
            if(j < len(downEdges[i]) - 1):
                textJSON += ","
        textJSON += "]";
        textJSON += "}\n" if i == nAlignmentNodes - 1 else "},\n"

    textJSON += "  ],\n"

    textJSON += "  \"edges\": [\n"

    for i in range(0,nAlignmentEdges):
        textJSON += "    {"
        id1 = alignment_vtk.GetCell(i).GetPointId(0)
        id2 = alignment_vtk.GetCell(i).GetPointId(1)
        textJSON += "\"node1\": " + str(id1) + ", \"node2\": " + str(id2)
        textJSON += "}\n" if i == nAlignmentEdges - 1 else "},\n"

    textJSON += "  ]\n"

    textJSON += "}\n"
    
    res.append(textJSON)
    
    #==================================================================================================================
    # output original trees as JSON

    for t in range(0,n):
                        
        upEdges = []
        downEdges = []
        for i in range(0,nVertices[t]):
            upEdges.append([])
            downEdges.append([])

        for i in range(input_MB.GetBlock(t).GetNumberOfCells()):
            id1 = input_MB.GetBlock(t).GetCell(i).GetPointId(0)
            id2 = input_MB.GetBlock(t).GetCell(i).GetPointId(1)
            v1 = input_MB.GetBlock(t).GetPointData().GetArray("Scalar").GetValue(id1)
            v2 = input_MB.GetBlock(t).GetPointData().GetArray("Scalar").GetValue(id2)
            if(v1 > v2):
                downEdges[id1].append(i)
                upEdges[id2].append(i)
            else:
                upEdges[id1].append(i)
                downEdges[id2].append(i)

        textJSON = "{\n"

        textJSON += "  \"nodes\": [\n"

        first = True
        for k in range(0,nAlignmentNodes):
            i = int(vertexIDs.GetComponent(k, t))
            if(i < 0):
                continue
            textJSON += "    {" if first else ",\n    {"
            textJSON += "\"scalar\": " + str(input_MB.GetBlock(t).GetPointData().GetArray("Scalar").GetValue(i)) + ", "
            textJSON += "\"id\": " + str(alignmentIDs[t][i]) + ", "
            textJSON += "\"upEdgeIDs\": [";
            for j in range(0,len(upEdges[i])):
                textJSON += str(upEdges[i][j])
                if(j < len(upEdges[i]) - 1):
                    textJSON += ","
            textJSON += "], "
            textJSON += "\"downEdgeIDs\": ["
            for j in range(len(downEdges[i])):
                textJSON += str(downEdges[i][j])
                if(j < len(downEdges[i]) - 1):
                    textJSON += ","
            textJSON += "]"
            textJSON += "}"
            first = False

        textJSON += "\n  ],\n"

        textJSON += "  \"edges\": [\n"

        for i in range(0,input_MB.GetBlock(t).GetNumberOfCells()):
            textJSON += "    {";
            id1 = input_MB.GetBlock(t).GetCell(i).GetPointId(0)
            id2 = input_MB.GetBlock(t).GetCell(i).GetPointId(1)
            #textJSON += "\"node1\": " + str(id1) + ", \"node2\": " + str(id2)
            textJSON += "\"node1\": " + str(alignmentIDs[t][id1]) + ", \"node2\": " + str(alignmentIDs[t][id2])
            textJSON += "}\n" if i == input_MB.GetBlock(t).GetNumberOfCells() - 1 else "},\n"

        textJSON += "  ]\n"

        textJSON += "}\n"
        
        res.append(textJSON)
    
    return res


res = alignmentToJSON()

res2 = []
for i in range(len(res)):
    res2.append(json.loads(res[i]))
#print(res2)

with open('sim1.json', 'w') as json_file:
    json.dump(res2, json_file, indent=4)

#print(len(res))

#for s in res:
    #print(s)

28
27
10
[16, 20, 18, 14, 12, 12, 16, 12, 12, 14]
vtkLongLongArray (0x666d1590)
  Debug: Off
  Modified Time: 1027038
  Reference Count: 9
  Registered Events: (none)
  Name: VertexIDs
  Data type: long long
  Size: 280
  MaxId: 279
  NumberOfComponents: 10
  Information: 0x416ca3e0
    Debug: Off
    Modified Time: 1030926
    Reference Count: 1
    Registered Events: (none)
    PER_FINITE_COMPONENT: vtkInformationVector(0x44e08ca0)
    L2_NORM_FINITE_RANGE: 2.64575 41.9643
    PER_COMPONENT: vtkInformationVector(0x3bd194c0)
    L2_NORM_RANGE: 2.64575 41.9643
  Name: VertexIDs
  Number Of Components: 10
  Number Of Tuples: 28
  Size: 280
  MaxId: 279
  LookupTable: (none)




# Layout Computation

In the following, the layout for the contour tree alignment is computed. The layout algorithm allows an intuitive
joint depiction of multiple contour trees in a sensible manner – the *fuzzy contour tree*. The alignment, which is described above, is used to achieve a layout for each of the individual contour trees, which are then combined into the fuzzy contour tree.

In order to achieve a high recognition factor for the fuzzy contour
tree, we use the well-established and often-used orthogonal layout
as a basis for our algorithm. Finding an orthogonal layout for the
alignment (and thus for all aligned contour trees) is done in analogy
to finding a layout for (single) contour trees.

The first step is to recursively compute a branch decomposition. The resulting branches are then assigned horizontal positions. The vertical position of their nodes is given by the nodes' isovalues. The resulting, equally layouted individual contour (sub-)trees of the alignment are then combined.

The process is described in more detail in the following, accompanying the according code.    
**To show or hide the function code, click the *Toggle ...* button.**

In [12]:
def getNodeByID(tree, nodeid):
    if('frequency' in tree['nodes'][0]): # in alignment
        if(nodeid < len(tree['nodes'])):
            return tree['nodes'][nodeid]
    
    for i in range(len(tree['nodes'])):  # in trees
        if(tree['nodes'][i]['id'] == nodeid):
            return tree['nodes'][i]
f('getNodeByID')

In [13]:
def checkAlignment(alignment, trees, gAmaxscalar, gAminscalar):
    if(not 'id' in alignment['nodes']):
        for n in range(len(alignment['nodes'])):
            alignment['nodes'][n]['id'] = n
            
    for n in range(len(alignment['nodes'])):
        curn = alignment['nodes'][n]
        ttype = None
        if(gAmaxscalar == None or gAmaxscalar < alignment['nodes'][n]['scalar']):
            gAmaxscalar = alignment['nodes'][n]['scalar']
        if(gAminscalar == None or gAminscalar > alignment['nodes'][n]['scalar']):
            gAminscalar = alignment['nodes'][n]['scalar']
        if(len(curn['upEdgeIDs']) + len(curn['downEdgeIDs']) > 1): # check inner nodes of alignment
            parentids = []
            for t in range(len(trees)):
                curtn = getNodeByID(trees[t], n)
                if(curtn != None):
                    if(len(curtn['upEdgeIDs']) + len(curtn['downEdgeIDs']) > 1):
                        if(ttype == 'leaf'):
                            ttype = 'both'
                        else:
                            ttype = 'saddle'
                    else:
                        parent = None
                        if(len(curtn['upEdgeIDs']) == 1):
                            parent = alignment['edges'][curtn['upEdgeIDs'][0]]['node1']
                            if(parent == n):
                                parent = alignment['edges'][curtn['upEdgeIDs'][0]]['node2']
                        else:
                            parent = alignment['edges'][curtn['downEdgeIDs'][0]]['node1']
                            if(parent == n):
                                parent = alignment['edges'][curtn['downEdgeIDs'][0]]['node2']
                        if(not parent in parentids):
                            parentids.append(parent)
                        if(ttype == 'saddle'):
                            ttype = 'both'
                        else:
                            ttype = 'leaf'
                    if(ttype == 'both'):
                        raise ValueError('leaf and saddle are matched on the same alignment node nr '+n+'. this is not good.')
                if(ttype == 'leaf'):
                    print('pull out leaf')
                    neighborids = []
                    parentid = None
                    edgeids = curn['upEdgeIDs'] + curn['downEdgeIDs']
                    for e in range(len(edgeids)):
                        neighbor = alignment['edges'][edgeids[e]]['node1']
                        if(neighbor == n):
                            neighbor = alignment['edges'][edgeids[e]]['node2']
                        if(neighbor in parentids):
                            if(parentid == None):
                                parentid = neighbor
                            else:
                                raise ValueError('found different parent nodes as neighbor of ', n)
                        else:
                            neighborids.append(neighbor)
                    
                    # change connections
                    for e in range(len(edgeids)):
                        curedge = alignment['edges'][edgeids[e]]
                        if(curedge['node2'] == n):
                            curedge['node2'] = curedge['node1']
                        if(curedge['node2'] != parentid):
                            curedge['node1'] = parentid
                            
                            # check if new edge is up or down
                            if(alignment['nodes'][curedge['node1']]['scalar'] < alignment['nodes'][curedge['node2']]['scalar']):
                                if(edgeids[e] in alignment['nodes'][curedge['node1']]['upEdgeIDs']):
                                    alignment['nodes'][curedge['node1']]['upEdgeIDs'].append(edgeids[e])
                                    if(edgeids[e] in alignment['nodes'][curedge['node1']]['downEdgeIDs']):
                                        alignment['nodes'][curedge['node1']]['downEdgeIDs'].remove(edgeids[e])
                                if(edgeids[e] in alignment['nodes'][curedge['node2']]['downEdgeIDs']):
                                    alignment['nodes'][curedge['node2']]['downEdgeIDs'].append(edgeids[e])
                                    if(edgeids[e] in alignment['nodes'][curedge['node2']]['upEdgeIDs']):
                                        alignment['nodes'][curedge['node2']]['upEdgeIDs'].remove(edgeids[e])
                            else:
                                if(edgeids[e] in alignment['nodes'][curedge['node2']]['upEdgeIDs']):
                                    alignment['nodes'][curedge['node2']]['upEdgeIDs'].append(edgeids[e])
                                    if(edgeids[e] in alignment['nodes'][curedge['node2']]['downEdgeIDs']):
                                        alignment['nodes'][curedge['node2']]['downEdgeIDs'].remove(edgeids[e])
                                if(edgeids[e] in alignment['nodes'][curedge['node1']]['downEdgeIDs']):
                                    alignment['nodes'][curedge['node1']]['downEdgeIDs'].append(edgeids[e])
                                    if(edgeids[e] in alignment['nodes'][curedge['node1']]['upEdgeIDs']):
                                        alignment['nodes'][curedge['node1']]['upEdgeIDs'].remove(edgeids[e])
                            
                            if(edgeids[e] in alignment['nodes'][n]['upEdgeIDs']):
                                alignment['nodes'][n]['upEdgeIDs'].remove(alignment['nodes'][n]['upEdgeIDs'][edgeids[e]])
                            else:
                                if(edgeids[e] in alignment['nodes'][n]['downEdgeIDs']):
                                    alignment['nodes'][n]['downEdgeIDs'].remove(alignment['nodes'][n]['downEdgeIDs'][edgeids[e]])
            return gAmaxscalar, gAminscalar
f('checkAlignment')

The function *findPath()* is used to find a path from a startnode (the root node or a saddle node) to a stopnode (a leaf). Separate for monotonous increasing and decreasing direction, the candidate path from startnode to stopnode, which is a path in the alignment, is then considered in each individual tree, and counted if it is monotone, giving its *path frequency F*.

In [14]:
def findPath(tree, alignment, start, stop, mode=None, path=[], touched=[]):    
    path.append(start)
    touched[start['id']] = True
    edgestodo = []
    
    if(mode == 'up' or mode == None):
        for e in range(len(start['upEdgeIDs'])):
            edgestodo.append(tree['edges'][start['upEdgeIDs'][e]])
    if(mode == 'down' or mode == None):
        for e in range(len(start['downEdgeIDs'])):
            edgestodo.append(tree['edges'][start['downEdgeIDs'][e]])
    while(len(edgestodo) > 0):
        e = edgestodo.pop(0)
        n = getNodeByID(tree, e['node1'])
        
        if( n == None):
            continue
        if(n['id'] == start['id']):
            n = getNodeByID(tree, e['node2'])
        if('fixed' in getNodeByID(alignment, n['id'])):
            continue
        if(n['id'] == stop['id']):
            path.append(stop)
            return path
        if(not touched[n['id']]):
            pathcopy = path.copy()
            foundpath = findPath(tree, alignment, n, stop, mode, pathcopy, touched)
            if(foundpath != None):
                return foundpath
f('findPath')

The function *findMainBranch()* is called from the *getBranches()* function, which can be found further below, and serves to compute the branch for a given root or saddle node. 
Starting with the designated root node, a main branch is chosen by considering
both, alignment and individual contour trees, as follows:
All possible paths in the alignment from the root node to each leaf
are initially considered as candidates for the main branch.
Choosing the path with the highest rating *R* (which is based on its *path frequency* and *path persistency*) as the main branch
and proceeding recursively for each sub-branch (i.e. saddle) of the
main branch yields a branch decomposition for the alignment.

In [15]:
def findMainBranch(trees, alignment, root, maxscalar, minscalar, freq_w, pers_w):
    modes = ['up', 'down']
    moderesults = []
    
    if(not 'id' in root):
        print('no id in root')
        return
    
    for m in range(len(modes)):
        mode = modes[m]
        mstopnode = None
        mmaxRating = None
        malignment_path = None
        
        for n in range(len(alignment['nodes'])):
            curalignment_path = None
            if(len(alignment['nodes'][n]['upEdgeIDs']) + len(alignment['nodes'][n]['downEdgeIDs']) == 1 and not ('fixed' in alignment['nodes'][n])):
                touched = [None] * len(alignment['nodes'])
                curalignment_path = findPath(alignment, alignment, start=root, stop=alignment['nodes'][n], path=[], touched = touched)
                if(curalignment_path != None):
                    frequency = 0
                    for t in range(len(trees)):
                        start = getNodeByID(trees[t], root['id'])
                        stop  = getNodeByID(trees[t], n)                        
                        touched = [None] * len(alignment['nodes'])
                        if(start == None or stop == None):
                            continue                        
                        if(findPath(trees[t], alignment, start, stop, mode, touched = touched) != None):
                            frequency += 1
                    
                    if(frequency > 0):
                        persistencePercent = abs(root['scalar'] - alignment['nodes'][n]['scalar']) * 100.0/abs(maxscalar - minscalar)
                        frequencyPercent   = frequency * 100.0/len(trees)
                        currating = persistencePercent * pers_w + frequencyPercent * freq_w
                    
                        if(mmaxRating == None or currating > mmaxRating):
                            mmaxRating = currating
                            mstopnode = alignment['nodes'][n]
                            malignment_path = curalignment_path
                else:
                    continue
            else:
                continue
            if(mmaxRating == 100):
                break
        moderesults.append([mstopnode, mmaxRating, mode, malignment_path])
        
    stopnode       = moderesults[0][0]
    maxRating      = moderesults[0][1]
    mode           = moderesults[0][2]
    alignment_path = moderesults[0][3]

    if(len(moderesults) > 1):
        for m in range(len(moderesults)):
            if(maxRating != None and moderesults[m][1] != None):
                if(moderesults[m][1] > maxRating):
                    stopnode       = moderesults[m][0]
                    maxRating      = moderesults[m][1]
                    mode           = moderesults[m][2]
                    alignment_path = moderesults[m][3]
            elif(moderesults[m][1] != None):
                stopnode       = moderesults[m][0]
                maxRating      = moderesults[m][1]
                mode           = moderesults[m][2]
                alignment_path = moderesults[m][3]

    if(stopnode == None):
        return
    if(alignment_path == None):
        print("al path is None!")

    isoRange = None
    saddleIsoRange = None
    nodeIsoRange = None

    for t in range(len(trees)):            
        tr = getNodeByID(trees[t], root['id'])
        ts = getNodeByID(trees[t], stopnode['id'])

        if(tr != None or ts != None):
            if(tr != None and ts != None):
                if(isoRange == None):
                    isoRange = [min(tr['scalar'], ts['scalar']), max(tr['scalar'], ts['scalar'])]
                else:
                    isor = [min(tr['scalar'], ts['scalar']), max(tr['scalar'], ts['scalar'])]
                    if (isoRange[0] > isor[0]):
                        isoRange[0] = isor[0]
                    if (isoRange[1] < isor[1]):
                        isoRange[1] = isor[1]
                if(saddleIsoRange == None):
                    saddleIsoRange = [tr['scalar'], tr['scalar']]
                else:
                    if (saddleIsoRange[0] > tr['scalar']):
                        saddleIsoRange[0] = tr['scalar']
                    if (saddleIsoRange[1] < tr['scalar']):
                        saddleIsoRange[1] = tr['scalar']
                if(nodeIsoRange == None):
                    nodeIsoRange = [ts['scalar'], ts['scalar']]
                else:
                    if (nodeIsoRange[0] > ts['scalar']):
                        nodeIsoRange[0] = ts['scalar']
                    if (nodeIsoRange[1] < ts['scalar']):
                        nodeIsoRange[1] = ts['scalar']
            elif(tr != None):
                if(isoRange == None):
                    isoRange = [tr['scalar'], tr['scalar']]
                else:
                    isor = [tr['scalar'], tr['scalar']]
                    if (isoRange[0] > isor[0]):
                        isoRange[0] = isor[0]
                    if (isoRange[1] < isor[1]):
                        isoRange[1] = isor[1]
                if(saddleIsoRange == None):
                    saddleIsoRange = [tr['scalar'], tr['scalar']]
                else:
                    if (saddleIsoRange[0] > tr['scalar']):
                        saddleIsoRange[0] = tr['scalar']
                    if (saddleIsoRange[1] < tr['scalar']):
                        saddleIsoRange[1] = tr['scalar']
            elif(ts != None):
                if(isoRange == None):
                    isoRange = [ts['scalar'], ts['scalar']]
                else:
                    if (isoRange[0] > ts['scalar']):
                        isoRange[0] = ts['scalar']
                    if (isoRange[1] < ts['scalar']):
                        isoRange[1] = ts['scalar']
                if(nodeIsoRange == None):
                    nodeIsoRange = [ts['scalar'], ts['scalar']]
                else:
                    if (nodeIsoRange[0] > ts['scalar']):
                        nodeIsoRange[0] = ts['scalar']
                    if (nodeIsoRange[1] < ts['scalar']):
                        nodeIsoRange[1] = ts['scalar']
    return [root, stopnode, mode, maxRating, isoRange, saddleIsoRange, nodeIsoRange, alignment_path]

f('findMainBranch')
                                            

The function *findMainZeroFreqBranch()* is called from *getBranches()* in case no valid branch could be found with *findMainBranch()*. This occurs if no contour tree contains a path from the currently considered saddle to any leaf. The frequency of this branch is then considered zero and the rating is based solely on the path persistence, regardless of the chosen frequency/persistency weights.

In [33]:
def findMainZeroFreqBranch(trees, alignment, root, maxscalar, minscalar, freq_w, pers_w):
    maxRating = None
    stopnode = None
    alignment_path = None
    
    for n in range(len(alignment['nodes'])):
        curalignment_path = None
        if(len(alignment['nodes'][n]['upEdgeIDs']) + len(alignment['nodes'][n]['downEdgeIDs']) == 1 
           and not('fixed' in alignment['nodes'][n])):
            #print(alignment['nodes'][n],"\n\n")
            touched = [None] * len(alignment['nodes'])
            curalignment_path = findPath(alignment, alignment, start=root, stop=alignment['nodes'][n], path = [], touched = touched)
            if(curalignment_path != None):
                currating = abs(root['scalar'] - alignment['nodes'][n]['scalar'])
                if(maxRating == None or currating > maxRating):
                    maxRating = currating
                    stopnode = alignment['nodes'][n]
                    alignment_path = curalignment_path
    
    mode = 'up'
    if(stopnode['scalar'] < root['scalar']):
        mode = 'down'
        
    isorange = None
    saddleIsorange = None
    nodeIsoRange = None
    noNodeRange = True
    
    for t in range(len(trees)):
        ts = getNodeByID(trees[t], stopnode['id'])
        tr = getNodeByID(trees[t], root['id'])
        
        if(tr == None and ts == None):
            continue
        
        if(tr != None and ts != None):
            if(isoRange == None):
                isoRange = [min(tr['scalar'], ts['scalar']), max(tr['scalar'], ts['scalar'])]
            else:
                isor = [min(tr['scalar'], ts['scalar']), max(tr['scalar'], ts['scalar'])]
                if (isoRange[0] > isor[0]):
                    isoRange[0] = isor[0]
                if (isoRange[1] < isor[1]):
                    isoRange[1] = isor[1]
            if(saddleIsoRange == None):
                saddleIsoRange = [tr['scalar'], tr['scalar']]
            else:
                if (saddleIsoRange[0] > tr['scalar']):
                    saddleIsoRange[0] = tr['scalar']
                if (saddleIsoRange[1] < tr['scalar']):
                    saddleIsoRange[1] = tr['scalar']
            if(nodeIsoRange == None):
                nodeIsoRange = [ts['scalar'], ts['scalar']]
            else:
                if (nodeIsoRange[0] > ts['scalar']):
                    nodeIsoRange[0] = ts['scalar']
                if (nodeIsoRange[1] < ts['scalar']):
                    nodeIsoRange[1] = ts['scalar']
        
        elif(tr != None):
            if(isorange == None):
                isorange = [tr['scalar'], tr['scalar']]
            else:
                if (isorange[0] > tr['scalar']):
                    isorange[0] = tr['scalar']
                if (isorange[1] < tr['scalar']):
                    isorange[1] = tr['scalar']
            if(saddleIsorange == None):
                saddleIsorange = [tr['scalar'], tr['scalar']]
            else:
                if (saddleIsorange[0] > tr['scalar']):
                    saddleIsorange[0] = tr['scalar']
                if (saddleIsorange[1] < tr['scalar']):
                    saddleIsorange[1] = tr['scalar']
        elif(ts != None):
            if(isorange == None):
                isorange = [ts['scalar'], ts['scalar']]
            else:
                if (isorange[0] > ts['scalar']):
                    isorange[0] = ts['scalar']
                if (isorange[1] < ts['scalar']):
                    isorange[1] = ts['scalar']
            if(nodeIsoRange == None):
                nodeIsoRange = [ts['scalar'], ts['scalar']]
            else:
                if (nodeIsoRange[0] > ts['scalar']):
                    nodeIsoRange[0] = ts['scalar']
                if (nodeIsoRange[1] < ts['scalar']):
                    nodeIsoRange[1] = ts['scalar']

    return [root, stopnode, mode, maxRating* 100.0/abs(maxscalar-minscalar) * pers_w, isorange, saddleIsorange, nodeIsoRange, alignment_path]
    
f('findMainZeroFreqBranch')

The function *getBranches()* initiates the recursive branch decomposition by calling *findMainBranch()* or *findMainZeroFreqBranch()* first for the root node of the alignment and then for every saddle node which occurs in a newly found branch. The computed alignment provides a dedicated root node, which means we can use this node as a starting point for the main branch and choose the rest of the branch considering paths in both the alignment and the individual trees, based on their frequency and persistency. A rating R is given for each candidate path based on the following formula:

$$R = P_{\%} \cdot w_{pers} + F_{\%} \cdot w_{freq}$$

where $P_{\%}$ is the percentage of the path persistence relative to the value range $[I_{min}, I_{max}]$ of the alignment $P_{\%} = \frac{P \cdot 100}{I_{max} - I_{min}}$ and $F_{\%}$ is the percentage of the n contour trees that contain the considered path $F_{\%} = \frac{F \cdot 100}{n}$. $w_{pers}$ and $w_{freq}$ are the user chosen weights for persistency and frequency, which can be adjusted with the weight slider down below. The candidate path with highest rating R is then chosen as the branch and the nodes on the path between start- and endnode are the new startnodes (saddle nodes) for a new branch.

In [17]:
def getBranches(trees, alignment, gAmaxscalar, gAminscalar, freq_w, pers_w):
    root = alignment['nodes'][0]
    for n in range(len(alignment['nodes'])):
        if(len(alignment['nodes'][n]['upEdgeIDs']) + len(alignment['nodes'][n]['downEdgeIDs']) > 1):
            continue
        if(gAmaxscalar == None):
            gAmaxscalar = alignment['nodes'][n]['scalar']
        elif(gAmaxscalar < alignment['nodes'][n]['scalar']):
            gAmaxscalar = alignment['nodes'][n]['scalar']
        if(gAminscalar == None):
            gAminscalar = alignment['nodes'][n]['scalar']
        elif(gAminscalar > alignment['nodes'][n]['scalar']):
            gAminscalar = alignment['nodes'][n]['scalar']
        
        startnodes = [{'node': root, 'parent': -1}]
        branches = []
        
        counter = 0
        while(len(startnodes) > 0):
            startnode = startnodes.pop(0)            
            start = startnode['node']
            parent = startnode['parent']
            
            bestpath = findMainBranch(trees, alignment, start, gAmaxscalar, gAminscalar, freq_w, pers_w)
            if(bestpath == None):              
                if(start == root):
                    print("no main branch for root found, break")
                    break
                print("no branch found, try main zero freq branch for ", counter)
                bestpath = findMainZeroFreqBranch(trees, alignment, start, gAmaxscalar, gAminscalar, freq_w, pers_w)
                counter += 1
            
            if(parent >= 0):
                branches.append({'nodes': bestpath[7], 'mode': bestpath[2], 'parentBranch': parent, 'depth': branches[parent]['depth']+1, 'rating': bestpath[3], 'isorange': bestpath[4], 'saddleIsorange': bestpath[5], 'nodeIsorange': bestpath[6]})
            else: # main branch
                if(bestpath[2] == 'down'):
                    nodeIsoRange = None
                    for t in range(len(trees)):
                        ts = getNodeByID(trees[t], bestpath[0]['id'])
                        if(ts != None):
                            if(nodeIsoRange == None):
                                nodeIsoRange = [ts['scalar'], ts['scalar']]
                            else:
                                if (nodeIsoRange[0] > ts['scalar']):
                                    nodeIsoRange[0] = ts['scalar']
                                if (nodeIsoRange[1] < ts['scalar']):
                                    nodeIsoRange[1] = ts['scalar']
                    bestpath[7][len(bestpath[7])-1]['root_id'] = bestpath[7][0]['id']
                    # list[::-1] reverses the list
                    branches.append({'nodes': bestpath[7][::-1], 'mode': 'up', 'parentBranch': parent, 'depth': 0, 'rating': bestpath[3], 'isorange': bestpath[4], 'saddleIsorange': None, 'nodeIsorange': nodeIsoRange})
                else:
                    bestpath[7][0]['root_id'] = bestpath[7][len(bestpath[7])-1]['id']
                    branches.append({'nodes': bestpath[7], 'mode': bestpath[2], 'parentBranch': parent, 'depth': 0, 'rating': bestpath[3], 'isorange': bestpath[4], 'saddleIsorange': None, 'nodeIsorange': bestpath[6]})
            
            bestpath[7][0]['fixed'] = True
            bestpath[7][-1]['fixed'] = True
            
            for s in range(1, len(bestpath[7])-1):
                bestpath[7][s]['fixed'] = True
                startnodes.append({'node': bestpath[7][s], 'parent': len(branches)-1})
            
            startnodeedges = start['upEdgeIDs'] + start['downEdgeIDs']
            for e in range(len(startnodeedges)):
                neighbor = alignment['edges'][startnodeedges[e]]['node1']
                if(neighbor == start['id']):
                    neighbor = alignment['edges'][startnodeedges[e]]['node2']
                neighbor = getNodeByID(alignment, neighbor)
                if(not ('fixed' in neighbor)):
                    startnodes.append({'node': start, 'parent': parent})
                    break
            counter += 1
        return branches
    
f('getBranches')

For each node in the alignment, the corresponding nodes in the trees are considered. These nodes are matched, but they still might have different scalar values. This is important, since the vertical position of a node is derived from the scalar value of this node. If a leaf node appears in more than one tree and has different scalar values, it is drawn multiple times, once for each scalar value. Saddle nodes are not explicitly drawn, but their scalar values determine where the edge representing its branch starts.

In the grouped layout, horizontal edges for every scalar value a saddle node can have are drawn. In the bundled layout, the horizontal edges of a branch are bundled into one edge. A beak at the position of the saddle node depicts the range of scalar values the saddle has in different trees. The function *getSaddleMean()* serves to find the mean scalar value for each saddle, which will be where the horizontal edge for the bundled layout is drawn.

In [18]:
def getSaddleMean(branches, trees, alignment):
    # add depth to nodes of branches
    for b in range(len(branches)):
        for n in range(len(branches[b]['nodes'])):
            if(not 'depth' in branches[b]['nodes'][n]):
                branches[b]['nodes'][n]['depth'] = branches[b]['depth']

    # add depth to nodes of trees
    for t in range(len(trees)):
        for n in range(len(trees[t]['nodes'])):
            trees[t]['nodes'][n]['depth'] = alignment['nodes'][trees[t]['nodes'][n]['id']]['depth']
        
    for b in range(1, len(branches)):
        an1 = branches[b]['nodes'][0]
        an2 = branches[b]['nodes'][-1]
        
        if(branches[b]['saddleIsorange'] == None):
            print('no saddleIsorange for branch ', b)
            continue
        
        if(an1['depth'] > an2['depth']): # swap
            an1, an2 = an2, an1
        
        saddleMean = 0
        leafMean   = 0
        saddles    = []
        leaves     = []
        strees     = []
        leafmaxs   = None
        leafmins   = None
                
        if(an1['scalar'] != an2['scalar']):
            for t in range(len(trees)):
                tln = getNodeByID(trees[t], an2['id'])
                if(tln == None):
                    continue
                
                if((len(tln['upEdgeIDs']) + len(tln['downEdgeIDs'])) != 1): # an2 is no leaf
                    tln = getNodeByID(trees[t], an1['id'])
                    if(tln == None):
                        continue
                    if((len(tln['upEdgeIDs']) + len(tln['downEdgeIDs'])) != 1): # an1 is also no leaf
                        raise ValueError('neither startnode nor endnode of branch ', b,' are leafs')
                
                tsn = None
                mode = None
                if(len(tln['upEdgeIDs']) == 1):
                    mode = 'up'
                    tsn = getNodeByID(trees[t], trees[t]['edges'][tln['upEdgeIDs'][0]]['node2'])
                    if(tsn['id'] == tln['id']):
                        tsn = getNodeByID(trees[t], trees[t]['edges'][tln['upEdgeIDs'][0]]['node1'])
                else:
                    mode = 'down'
                    tsn = getNodeByID(trees[t], trees[t]['edges'][tln['downEdgeIDs'][0]]['node2'])
                    if(tsn['id'] == tln['id']):
                        tsn = getNodeByID(trees[t], trees[t]['edges'][tln['downEdgeIDs'][0]]['node1'])
                
                while(tsn['depth'] == tln['depth']):
                    edgestodo = None
                    if(mode == 'up'):
                        edgestodo = tsn['upEdgeIDs']
                    else:
                        edgestodo = tsn['downEdgeIDs']
                    
                    if(len(tsn['upEdgeIDs']) > 0):
                        node1 = getNodeByID(trees[t], trees[t]['edges'][tsn['upEdgeIDs'][0]]['node1'])
                        node2 = getNodeByID(trees[t], trees[t]['edges'][tsn['upEdgeIDs'][0]]['node2'])
                    if(len(tsn['downEdgeIDs']) > 0):
                        node1 = getNodeByID(trees[t], trees[t]['edges'][tsn['downEdgeIDs'][0]]['node1'])
                        node2 = getNodeByID(trees[t], trees[t]['edges'][tsn['downEdgeIDs'][0]]['node2'])
                    
                    node = None
                    
                    for e in range(len(edgestodo)):
                        node = getNodeByID(trees[t], trees[t]['edges'][edgestodo[e]]['node2'])
                        if(node['id'] == tsn['id']):
                            node = getNodeByID(trees[t], trees[t]['edges'][edgestodo[e]]['node1'])
                        if(node['depth'] != tsn['depth']):
                            break
                    
                    if(node == None):
                        raise ValueError('could not trace branch correctly')
                    
                    tsn = node
                
                saddleMean = saddleMean + tsn['scalar']
                leafMean   = leafMean   + tln['scalar']
                saddles.append(tsn)
                leaves.append(tln)
                strees.append(t)
                
                if(leafmaxs == None or leafmaxs < tln['scalar']):
                    leafmaxs = tln['scalar']
                if(leafmins == None or leafmins > tln['scalar']):
                    leafmins = tln['scalar']
                
                if (branches[b]['saddleIsorange'][0] > tsn['scalar']):
                    branches[b]['saddleIsorange'][0] = tsn['scalar']
                if (branches[b]['saddleIsorange'][1] < tsn['scalar']):
                    branches[b]['saddleIsorange'][1] = tsn['scalar']
            
        else:
            saddleMean = an1['scalar']
            saddles.append(an1)
            leafMean = an1['scalar']
            leaves.append(an1)
            leafmaxs = an1['scalar']
            leafmins = an1['scalar']
            
        
        
        if(saddles != None and len(saddles) > 0):
            saddleMean = saddleMean/len(saddles)
        if(leaves != None and len(leaves) > 0):
            leafMean   = leafMean/len(leaves)
        
        if(branches[b]['mode'] == 'up'):
            if(leafmins < saddleMean):
                saddleMean = leafmins
        else:
            if(leafmaxs > saddleMean):
                saddleMean = leafmaxs
        
        if(saddleMean == None or leafMean == None or saddles == None or leafmaxs == None or leafmins == None):
            raise ValueError('null value in getSaddleMean!')
            
        
        testmaxsaddle = None
        testminsaddle = None
        for i in range(len(saddles)):
            if(testmaxsaddle == None or saddles[i]['scalar'] > testmaxsaddle):
                testmaxsaddle = saddles[i]['scalar']
            if(testminsaddle == None or saddles[i]['scalar'] < testminsaddle):
                testminsaddle = saddles[i]['scalar']
                        
        
        branches[b]['saddleMean'] = saddleMean
        branches[b]['leafMean']   = leafMean
        branches[b]['saddles']    = saddles
        branches[b]['leaves']     = leaves
        branches[b]['leafmaxs']   = leafmaxs
        branches[b]['leafmins']   = leafmins
        branches[b]['strees']     = strees 
        
f('getSaddleMean')

Since in the bundled layout, the bundled edge of a branch *b* might not represent the full scalar range of its saddle node, a corner case can occur, where a saddle starting in *b* can have a scalar value (range) that is above a down-edge or below an up-edge, giving the impression that the branch starting in this saddle is floating without a connection to the alignment. These cases are detected in *handleLonelyBranches()* and the horizontal edge of a floating branch is extended to the beak of its parent, which represents the full scalar range of the saddle node of the parent.

In [19]:
def handleLonelyBranches(branches):
    # check if a branch has children that are outside of the drawing range for the bundled edge
    parent = None
    groups = []
    for b in range(len(branches)):
        if(branches[b]['parentBranch'] > 0):
            parent = branches[branches[b]['parentBranch']]
            if(parent['mode'] == 'up'):
                parentRange = [parent['saddleMean'], parent['isorange'][1]]
            else:
                parentRange = [parent['isorange'][0], parent['saddleMean']]
            
            if(branches[b]['saddleMean'] < parentRange[0] or branches[b]['saddleMean'] > parentRange[1]):
                if('members' in parent):
                    parent['members'].append(b)
                else:
                    members = [b]
                    parent['members'] = members
                branches[b]['isLonely'] = True
            else:
                branches[b]['groupID'] = len(groups) - 1
                groups.append(branches[b])
        else:
            branches[b]['groupID'] = len(groups) - 1
            groups.append(branches[b])
    return groups

f('handleLonelyBranches')

After a branch decomposition for the alignment is established,
many known layout algorithms for contour trees could be employed.
Here, the method proposed by Heine et al. in their permutation phase [1] is adapted: An
ordering of branch groups that minimizes the weighted number of
edge crossings is computed. Instead of branch persistence the rating
obtained during branch decomposition as weight is deployed. Thus, branches
that have been chosen as main branches for the whole alignment
or sub-trees are less likely to be crossed in the resulting ordering.
Finding the optimum ordering is achieved as proposed by Heine et
al. by using a random walk in combination with simulated annealing.

After an ordering is found, giving each branch its own horizontal position, it is checked for neighboring branches if they can share a horizontal position, which is the case if their vertical ranges do not overlap, resulting in a more concise layout.

<sub>[1]:C. Heine, D. Schneider, H. Carr and G. Scheuermann, "Drawing Contour Trees in the Plane," in IEEE Transactions on Visualization and Computer Graphics, vol. 17, no. 11, pp. 1599-1611, Nov. 2011, doi: 10.1109/TVCG.2010.270.</sub>

In [20]:
def cost(branchGroups, permutation, max_cost = None):
    cost = 0
    for b in range(len(branchGroups)):
        ib = permutation[b]
        ip = None
        if(branchGroups[b]['parentBranch'] >= 0):
            tempBranch = branchGroups[b]
            if(not('groupID' in branches[tempBranch['parentBranch']])):
                #print(tempBranch['parentBranch'])
                tempBranch = branches[tempBranch['parentBranch']]
            ip = permutation[branches[tempBranch['parentBranch']]['groupID']]
        else:
            continue
        
        start = ib
        stop = ip
        if(ib > ip):
            start = ip
            stop = ib
        
        b2 = None
        for ib2 in range(start+1, stop-1):
            b2 = permutation.index(ib2)
            if(branchGroups[b]['saddleIsorange'][0] <= branchGroups[b2]['isorange'][1] and branchGroups[b2]['isorange'][0] <= branches[b]['saddleIsorange'][1]):
                cost = cost + branchGroups[b2]['rating']
                if(max_cost != None and cost > max_cost):
                    return cost
    return cost

def getLayout(branches, branchGroups, trees, alignment, usePredefinedLayout):
    permutation = [i for i in range(len(branchGroups))]
    random.shuffle(permutation)
    
    #main branch initially in the middle
    meanIndex = permutation.index(int(len(permutation)/2))
    temp = permutation[meanIndex]
    permutation[meanIndex] = permutation[0]
    permutation[0] = temp
    
    bestcost = cost(branchGroups, permutation)
    bestPermutation = permutation.copy()

    noBetterCount = 0
    i = 0
    tau = len(branchGroups) * 3
    
    while(noBetterCount < tau):
        shiftId = int(random.uniform(0, 1)*len(permutation)) # int always takes floor
        shiftToId = int(random.uniform(0, 1)*len(permutation))
        
        # punish if main branch is in the middle and will be moved
        shiftMainBranch = False
        if(permutation[0] == int(len(permutation)/2)): # main branch now in the middle
            if(shiftId == 0 or shiftToId == 0):
                shiftMainBranch = True
        
        if(shiftId != shiftToId):
            tmp = permutation[shiftId]
            permutation[shiftId] = permutation[shiftToId]
            permutation[shiftToId] = tmp
        curcost = cost(branchGroups, permutation)
        if(shiftMainBranch):
            curcost += branches[0]['rating']*4
        
        if(curcost < bestcost):
            bestcost = curcost
            bestPermutation = permutation.copy()
            noBetterCount = 0
        else:
            if(random.uniform(0, 1) >= math.exp(-(i*(abs(bestcost - curcost)))/tau)):
                tmp = permutation[shiftId]
                permutation[shiftId] = permutation[shiftToId]
                permutation[shiftToId] = tmp
            noBetterCount += 1
        i += 1
    
    print('found optimal permutation with cost ', bestcost)
    
    newPermutation = [None] * len(branches)
    curBranch = parent = parentslot = index1 = index2 = member = None
    shift = 0
    todo = []
    
    for slot in range(len(bestPermutation)):
        curBranch = branchGroups[bestPermutation.index(slot)]
        if('members' in curBranch):
            parent = curBranch['parentBranch']
            parentslot = bestPermutation[parent]
            index1 = branches.index(curBranch)
            if(parentslot < slot):
                newPermutation[index1] = slot + shift
                for m in range(len(curBranch['members'])):
                    shift += 1
                    index2 = curBranch['members'][m]
                    newPermutation[index2] = slot + shift
            else:
                for m in range(len(curBranch['members'])):
                    index2 = curBranch['members'][m]
                    newPermutation[index2] = slot + shift
                    shift += 1
                newPermutation[index1] = slot + shift
            member = branches[index1]
            if('members' in member):
                for i in range(len(member['members'])):
                    todo.append(branches[curBranch['members'][i]])
        else:
            index1 = branches.index(curBranch)
            newPermutation[index1] = slot + shift
    bestPermutation = newPermutation
    
    if(usePredefinedLayout and len(bestPermutation) == 12):
        bestPermutation = [6, 4, 3, 5, 1, 11, 9, 0, 7, 8, 10, 2]
    
    tempList = []
    for i in range(len(bestPermutation)):
        tempList.append(i)
    
    listwithnone = []
    for i in range(len(bestPermutation)):
        if(bestPermutation[i] == None):
            listwithnone.append(i)
        else:
            tempList.remove(bestPermutation[i])
    #print(tempList)
    #print(listwithnone)
    for i in range(len(listwithnone)):
        bestPermutation[listwithnone[i]] = tempList[i]
    
    # good permutation for time 3.5 with data:
    
    if(usePredefinedLayout):
        predefinedLayout = [3, 8, 7, 1, 0, 2, 6, 5, 9, 4]
        if(len(predefinedLayout) == len(bestPermutation)):
            bestPermutation = predefinedLayout
    
    print(bestPermutation)
    if(len(bestPermutation) != len(branches)):
        raise ValueError('Permutation has wrong length!')
        
    positions = []
    positions.append(0)
    branch = None
    slotRange = []
    newRange = []
    ranges = []
    
    branch = branches[bestPermutation.index(0)]
    if(branch['saddleIsorange'] != None): # not main branch
        saddleMean = branch['saddleMean']
        if(branch['mode'] == 'up'):
            slotRange.append([saddleMean, branch['isorange'][1]])
        else:
            slotRange.append([branch['isorange'][0], saddleMean])
        ranges.append(slotRange)
    else:
        ranges.append([[-math.inf, math.inf]])
    
    for slot in range(1, len(bestPermutation)):
        branch = branches[bestPermutation.index(slot)]
        if(branch['saddleIsorange'] == None): # main branch
            positions.append(positions[-1]+1)
            ranges.append([[-math.inf, math.inf]])
            continue
            
        saddleMean = branch['saddleMean']
        if(branch['mode'] == 'up'):
            newRange = [saddleMean, branch['isorange'][1]]
        else:
            newRange = [branch['isorange'][0], saddleMean]
        
        slotRange = ranges[-1]
        overlap = False
        for r in range(len(slotRange)):
            if(not newRange[0] > slotRange[r][1]):
                if(not newRange[1] < slotRange[r][0]):
                    positions.append(positions[-1] + 1)
                    ranges.append([newRange])
                    overlap = True
                    break
        if(not overlap):
            positions.append(positions[-1])
            ranges[-1].append(newRange)
    
    finalpos = positions.copy()
    for p in range(len(positions)):
        finalpos[bestPermutation.index(p)] = positions[p]
    
    step = 100/max(finalpos)+1
    if(len(branches) != len(finalpos)):
        raise ValueError('len(branches) != len(finalpos)')
    
    for b in range(len(branches)):
        branches[b]['x_percent'] = finalpos[b] * step
        

f('getLayout')

## Choose Parameters

In the following the frequency/persistency weights can be chosen.
The weights sum up to 1, so it suffices to choose only the frequency weight and derive the persistency weight from it.

In [21]:
output_checkbox_variable = widgets.Text()

def layout_assignment(Predefined_Layout):
    output_checkbox_variable.value = str(Predefined_Layout)

output_slider_variable = widgets.Text()

def weight_assignment(frequency):
    output_slider_variable.value = str(frequency)

f('slider')
    
interact(layout_assignment, Predefined_Layout=True)
interact(weight_assignment, frequency=(0.0,1.0,0.01))



interactive(children=(Checkbox(value=True, description='Predefined_Layout'), Output()), _dom_classes=('widget-…

interactive(children=(FloatSlider(value=0.5, description='frequency', max=1.0, step=0.01), Output()), _dom_cla…

<function __main__.weight_assignment(frequency)>

In the following, the preprocessing for the layout is done, i.e. the branch decomposition, saddle mean values and branch groups are computed.

In [41]:
# here the previously chosen parameters are retrieved
freq_w = round(float(output_slider_variable.value), 2)
pers_w = round(1-freq_w, 2)

if(output_checkbox_variable.value == 'True'):
    usePredefinedLayout = True
elif(output_checkbox_variable.value == 'False'):
    usePrettyLayout = False

# here the alignment and individual contour trees are retrieved from the alignment computation part
alignment = json.loads(res[0])
trees = []
for i in range(1, len(res)):
    tree = json.loads(res[i])
    trees.append(tree)

gAmaxscalar, gAminscalar = checkAlignment(alignment, trees, None, None)
branches = getBranches(trees, alignment, gAmaxscalar, gAminscalar, freq_w, pers_w)
getSaddleMean(branches, trees, alignment)
branchGroups = handleLonelyBranches(branches)
yshift = False

f('preprocessing')

no branch found, try main zero freq branch for  6


Now the layout can be computed:

In [42]:
getLayout(branches, branchGroups, trees, alignment, usePredefinedLayout)
# javascriptData is what is given to the d3 part which is used to visualize the result
javascriptData = [branches, trees, alignment, gAmaxscalar, gAminscalar, freq_w, pers_w]#, yShift]

found optimal permutation with cost  403.28268682878917
[7, 12, 0, 3, 5, 13, 2, 6, 9, 1, 11, 8, 10, 4]


In [43]:
HTML("""
<div id="alignInput"><input type="radio" id="b" name="layout-radio" value="bundle" checked><label for="b">Bundled Layout</label></div>
<div id="alignInput"><input type="radio" id="g" name="layout-radio" value="grouped"><label for="g">Grouped Layout</label></div>
<div id="alignInput"><input type="checkbox" id="myCheckbox"> yshift </div> </br>
<div id="svg" style="margin-top:30px;"> </div>
""")

In [44]:
Javascript("""
    var data = %s;
    var branches    = data[0];
    var trees       = data[1];
    var alignment   = data[2];
    var gAmaxscalar = data[3];
    var gAminscalar = data[4];
    var freq_w      = data[5];
    var pers_w      = data[6];
    var yshift      = false;
    var layoutStyle = 'bundle';

    require(['d3'], function (d3){
    $.getScript("draw.js")
        .done(function(){
            console.log("Loaded draw.js");
            (function(element){
                    var width = 1000;
                    var height = 600;
                    
                    d3.select("#myCheckbox").on("change",update);
                    //d3.select("input[name=layout-radio]:checked").on("change",update);
                    d3.select("#g").on("change",update);
                    d3.select("#b").on("change",update);
                    function update(){
                        //d3.select(element.get(0)).selectAll('*').remove();
                        d3.select("svg").remove();
                        if(d3.select("#myCheckbox").property("checked")){
                            yshift = true;
                        } else {
                            yshift = false;
                        }
                    
                        console.log(d3.select('input[name="layout-radio"]:checked').node().value);
                        layoutStyle = d3.select('input[name="layout-radio"]:checked').node().value;
                    
                    
                        var svg = d3.select("#svg").append('svg')
                            .attr("width", width)
                            .attr("height", height)
                            .attr("id", "alignment");
                        console.log(yshift)
                        draw(branches, trees, alignment, gAmaxscalar, gAminscalar, freq_w, pers_w, svg, width, height, yshift, layoutStyle);    
                    }
                    update();
            }(element));
        })
        .fail(function(){console.log("Unable to load draw.js")});
    })


    """ % json.dumps(javascriptData))

<IPython.core.display.Javascript object>